In [1]:
#SIGNS Dataset using python
#importing the various libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import cv2
import math
import h5py
import time
from sklearn.preprocessing import LabelBinarizer
%matplotlib inline
print("The tensorflow version is "+str(tf.__version__))
tf.reset_default_graph()

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The tensorflow version is 1.12.0


In [2]:
#Importing the dataset using pickle
def load_dataset():
     # Loading the data (cat/non-cat)
    train_dataset = h5py.File('train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # train set labels

    test_dataset = h5py.File('test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # test set labels
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
#loading the dataset
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes=load_dataset()
print("-x-x-x-x-x-x-x--x-x-x-x-x-x-x-x-Printing The Shapes-x-x-x-x-x-x-x-x-x-x-x-x-x--x")
print("The Training Shapes are for X {} and for Y {} ".format(train_set_x_orig.shape,train_set_y_orig.shape))
print("The Testing Shapes are for X {} and for Y {} ".format(test_set_x_orig.shape,test_set_y_orig.shape))

-x-x-x-x-x-x-x--x-x-x-x-x-x-x-x-Printing The Shapes-x-x-x-x-x-x-x-x-x-x-x-x-x--x
The Training Shapes are for X (1080, 64, 64, 3) and for Y (1080,) 
The Testing Shapes are for X (120, 64, 64, 3) and for Y (120,) 


In [4]:
#Reshaping them here
X_train=train_set_x_orig.reshape((train_set_x_orig.shape[0]),(train_set_x_orig.shape[1]*train_set_x_orig.shape[2]*train_set_x_orig.shape[3]))
X_test=test_set_x_orig.reshape((test_set_x_orig.shape[0]),(train_set_x_orig.shape[1]*train_set_x_orig.shape[2]*train_set_x_orig.shape[3]))
#One hot encoding the labels
lb=LabelBinarizer()
Y_train=lb.fit_transform(train_set_y_orig)
Y_test=lb.transform(test_set_y_orig)
print("The New Shapes are---")
print("X_train shape is {} and Y_train shape is  {}".format(X_train.shape,Y_train.shape))
print("X_test shape is {} and Y_test shape is  {}".format(X_test.shape,Y_test.shape))

The New Shapes are---
X_train shape is (1080, 12288) and Y_train shape is  (1080, 6)
X_test shape is (120, 12288) and Y_test shape is  (120, 6)


In [5]:
#Standardizng and shuffling the indices
X_train=X_train/255
X_test=X_test/255

In [ ]:
lr=[0.0003]
dropouts=[0.25]
for learning_rate in lr:
    for dropout in dropouts:
        model = Sequential()
        NAME="Model With {} dropout and {} layers at {} time @ learning rate of {}".format(dropout,128,int(time.time()),learning_rate)
        tensorboard=TensorBoard(log_dir="logs/{}".format(NAME))
        model.add(Flatten())
        model.add(Dense(128,activation=tf.nn.relu))
        model.add(Dense(128,activation=tf.nn.relu))
        model.add(Dense(128,activation=tf.nn.relu))
        model.add(Dropout(dropout))
        model.add(Dense(128,activation=tf.nn.relu))
        model.add(Dropout(dropout))
        model.add(Dense(128,activation=tf.nn.relu))
        model.add(Dense(6,activation=tf.nn.softmax))
        adamop=keras.optimizers.Adam(lr=learning_rate)
        model.compile(optimizer="adam",metrics=["accuracy"],loss="categorical_crossentropy")
        #install hdf5 package before proceeding ~pip install 
        #filepath="weights.best.hdf5"
        #checkpoint=ModelCheckpoint(filepath,monitor="val_acc",verbose=1,save_best_only=True,mode="max")
        callback_list=[tensorboard]
        model.fit(X_train,Y_train,shuffle=True,validation_split=0.1,batch_size=32,epochs=300,callbacks=callback_list)
        test_loss,test_acc=model.evaluate(X_test,Y_test)
        print("******** Testing Loss {} and Accuracy {} ****************".format(test_loss,test_acc))

Train on 972 samples, validate on 108 samples
Epoch 1/300
